## Mover aquí la gestión de nulos y outliers

Poner la clase del src y hacer una para outliers

In [2]:
import sys

sys.path.append("../")

from src.support_handling import impute_knn, MissingValuesHandler